#### xgb model that combines swing chase value, swing percent, and z-contact%

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import statsmodels.api as sm
import os 
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Active/Swing Decisons/')

import data

In [2]:
swing_data = pd.read_csv('./data/batter_data/plate_15_24.csv')
swing_value = pd.read_csv('./data/stat_values/swing_values.csv')

clean names for matching

In [3]:
import unicodedata

def clean_names(df, name_col):
    # Make a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Convert to lowercase and remove leading/trailing whitespace
    df[name_col] = df[name_col].str.lower().str.strip()
    
    # Remove accents
    df[name_col] = df[name_col].apply(lambda x: ''.join(
        c for c in unicodedata.normalize('NFKD', str(x))
        if not unicodedata.combining(c)
    ))
    
    # Keep only letters and replace multiple spaces with single space
    df[name_col] = df[name_col].str.replace('[^a-z ]', '', regex=True).str.replace('\s+', ' ', regex=True)
    
    return df

<>:17: SyntaxWarning: invalid escape sequence '\s'
<>:17: SyntaxWarning: invalid escape sequence '\s'
C:\Users\dalto\AppData\Local\Temp\ipykernel_27672\2103764149.py:17: SyntaxWarning: invalid escape sequence '\s'
  df[name_col] = df[name_col].str.replace('[^a-z ]', '', regex=True).str.replace('\s+', ' ', regex=True)


In [4]:
swing_data = clean_names(swing_data, 'Name')
swing_value = clean_names(swing_value, 'batter')
batter_ball = clean_names(pd.read_csv('./data/batter_data/batted_ball_2024.csv'), 'Name')

In [6]:
df = swing_data.merge(swing_value[['game_date','batter','chase_shadow_value', 'overall_value']], 
                      left_on=['Season','Name'], right_on=['game_date', 'batter'], how='left')

In [7]:
df = df.merge(batter_ball, on=['Name'], how='right')
df = df[df['chase_shadow_value'].notna()]
df = df[['batter', 'PA', 'game_date','chase_shadow_value', 'overall_value', 'Swing%', 'Contact%', 'whiff_percent']]
df = df[df['game_date'] == 2024]

In [8]:
Y_Chase = df['chase_shadow_value']
X_swing = df['Swing%']
X_whiff = df['whiff_percent']
X_swing = sm.add_constant(X_swing) # Add intercept

model_for_residuals = sm.OLS(Y_Chase, pd.concat([X_swing, X_whiff], axis=1))
results_for_residuals = model_for_residuals.fit()
       
print(results_for_residuals.summary())
df['adj_chase_shadow_value'] = results_for_residuals.resid


                            OLS Regression Results                            
Dep. Variable:     chase_shadow_value   R-squared:                       0.619
Model:                            OLS   Adj. R-squared:                  0.617
Method:                 Least Squares   F-statistic:                     393.7
Date:                Fri, 16 May 2025   Prob (F-statistic):          2.67e-102
Time:                        13:56:07   Log-Likelihood:                -1559.9
No. Observations:                 488   AIC:                             3126.
Df Residuals:                     485   BIC:                             3138.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           169.9742      2.595     65.510

In [10]:
Y_Chase = df['overall_value']
X_swing = df['Swing%']
X_whiff = df['whiff_percent']
X_swing = sm.add_constant(X_swing) # Add intercept

model_for_residuals = sm.OLS(Y_Chase, pd.concat([X_swing, X_whiff], axis=1))
results_for_residuals = model_for_residuals.fit()

        
print(results_for_residuals.summary())

df['adj_swing_value'] = results_for_residuals.resid 
correlation = df['adj_swing_value'].corr(df['Swing%'])
print(f"\nCorrelation {correlation:.4f}")

                            OLS Regression Results                            
Dep. Variable:          overall_value   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     15.87
Date:                Fri, 16 May 2025   Prob (F-statistic):           2.10e-07
Time:                        13:56:17   Log-Likelihood:                -1738.9
No. Observations:                 488   AIC:                             3484.
Df Residuals:                     485   BIC:                             3496.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           120.4089      3.745     32.154

In [11]:
df = df[df['PA'] > 25]
df = df[['batter','adj_chase_shadow_value', 'adj_swing_value']]

In [12]:
df.to_csv('./data/stat_values/swing_adj_data.csv')